In [ ]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 702.8/702.8 kB 7.0 MB/s eta 0:00:00


In [ ]:
import cv2
from ultralytics import YOLO
import numpy as np

model_path = '/content/drive/MyDrive/yolov8_segmentation_train_20_epochs/weights/best.pt'
image_path = '/content/drive/MyDrive/blending_logo_for_blue_car/input_frames/frame1.jpg'

# Read the image
img = cv2.imread(image_path)
H, W, _ = img.shape

# Load the YOLO model
model = YOLO(model_path)

# Perform inference
results = model(img)

# Initialize a blank mask
combined_mask = np.zeros((H, W), dtype=np.uint8)

# Iterate through each result and combine masks
for result in results:
    for j, mask in enumerate(result.masks.data):
        # Convert mask to appropriate format and resize
        mask = mask.numpy() * 255
        mask = cv2.resize(mask, (W, H))

        # Combine the current mask with the previous ones
        combined_mask = cv2.bitwise_or(combined_mask, mask.astype(np.uint8))

# Save the combined mask
cv2.imwrite('/content/drive/MyDrive/blending_logo_for_blue_car/mask.png', combined_mask)



0: 384x640 1 Logos, 261.3ms
Speed: 7.8ms preprocess, 261.3ms inference, 3.9ms postprocess per image at shape (1, 3, 384, 640)


True

In [ ]:
import cv2
import numpy as np

# Load the mask and logo images
mask_path = '/content/drive/MyDrive/blending_logo_for_blue_car/output_mask.png'
logo_path = '/content/drive/MyDrive/blending_logo_for_blue_car/logo_insta.png'

mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)  # Assuming mask is a grayscale image
logo = cv2.imread(logo_path, cv2.IMREAD_UNCHANGED)

# Find contours in the mask
contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Assuming the largest contour is the white region where the logo needs to be placed
contour = max(contours, key=cv2.contourArea)
x, y, w, h = cv2.boundingRect(contour)

# Resize logo to fit the white region
resized_logo = cv2.resize(logo, (w, h))

# Create a new blank image with white background
new_image = np.ones((mask.shape[0], mask.shape[1], 3), dtype=np.uint8) * 255

# Place the resized logo in the new image
if resized_logo.shape[2] == 4:  # Check if logo has transparency
    alpha_s = resized_logo[:, :, 3] / 255.0
    alpha_l = 1.0 - alpha_s

    for c in range(0, 3):
        new_image[y:y+h, x:x+w, c] = (alpha_s * resized_logo[:, :, c] +
                                      alpha_l * new_image[y:y+h, x:x+w, c])
else:
    new_image[y:y+h, x:x+w] = resized_logo

# Save the new image
cv2.imwrite('/content/drive/MyDrive/blending_logo_for_blue_car/resized_logo.png', new_image)


True

In [ ]:
import cv2
import numpy as np

# Load the image and mask
image = cv2.imread('/content/drive/MyDrive/data/images/train/trimmed-Abu-Dhabi-Grand-Prix-Logan-Sargeant-_mp4-0000_jpg.rf.416fc85c69cb0cba94c2ccb39c7fd755.jpg')
mask = cv2.imread('/content/drive/MyDrive/blending_logo_for_blue_car/output_mask.png', cv2.IMREAD_GRAYSCALE)

# Find contours in the mask
contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Create an empty mask of the same size as the input
result_mask = np.zeros_like(mask)

# Iterate through the contours and draw them on the result mask
cv2.drawContours(result_mask, contours, -1, (255), thickness=cv2.FILLED)

# Use the result mask to extract the region of interest from the original image
result_image = cv2.bitwise_and(image, image, mask=result_mask)

# Save the cropped image
cv2.imwrite('/content/drive/MyDrive/blending_logo_for_blue_car/cropped_img.png', result_image)


True

In [ ]:
from PIL import Image

# Load the base image
base_image_path = '/content/drive/MyDrive/data/images/train/trimmed-Abu-Dhabi-Grand-Prix-Logan-Sargeant-_mp4-0000_jpg.rf.416fc85c69cb0cba94c2ccb39c7fd755.jpg'
base_image = Image.open(base_image_path).convert('RGBA')

# Load the transparent image
transparent_image_path = 'blended_image_6.png'
transparent_image = Image.open(transparent_image_path).convert('RGBA')

# Paste the transparent image onto the base image
result_image = Image.alpha_composite(base_image, transparent_image)

# Display the result
result_image.show()

# Save the result
result_image.save('/content/drive/MyDrive/blending_logo_for_blue_car/res.png')
